# Reservoir Sampling distribué - énoncé

In [269]:
import pyensae
%nb_menu

## création d'un fichier à sampler

In [148]:
with open("sample4.txt", "w", encoding="utf8") as f:
    for i in range(0,100000):
        f.write("{0}\t{1}{0}\n".format(i, chr(i%26 + 65)))
    f.write("100001\tAAAAAA")

In [150]:
%head sample4.txt

## connexion

In [1]:
from pyquickhelper.ipythonhelper import open_html_form
params={"blob_storage":"", "password1":"", "hadoop_server":"", "password2":"", "username":"xavierdupre"}
open_html_form(params=params,title="server + hadoop + credentials", key_save="blobhp")

In [272]:
import pyensae
if False:
    blobstorage = blobhp["blob_storage"]
    blobpassword = blobhp["password1"]
    hadoop_server = blobhp["hadoop_server"]
    hadoop_password = blobhp["password2"]
    username = blobhp["username"]
    
blobstorage = "hdblobstorage"
blobpassword = "jQIPVO/T54w8X49UPIbzAVvaNO3wmuUwI4/o9AJnCaPTHoCQnsaGBUkT4eIyi0BRQavgc/TAQMQwy8eu19CSBQ=="
hadoop_server = "clusterensaeazure1"
hadoop_password = "2azureENSAE;"
username = "axavier"

In [273]:
client, bs = %hd_open
client, bs

(<pyensae.remote.azure_connection.AzureClient at 0x96426d8>,
 <azure.storage.blob.blobservice.BlobService at 0x9642ef0>)

## upload du fichier

In [151]:
%blob_up sample3.txt /$PSEUDO/sampling/sample4.txt

'$PSEUDO/sampling/sample4.txt'

In [152]:
%blob_ls /$PSEUDO/sampling

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/datafu-1.2.0.jar,"Fri, 13 Nov 2015 00:03:49 GMT",application/octet-stream,1600826,BlockBlob
1,axavier/sampling/sample.txt,"Fri, 13 Nov 2015 00:02:50 GMT",application/octet-stream,1377780,BlockBlob
2,axavier/sampling/sample2.txt,"Fri, 13 Nov 2015 00:35:55 GMT",application/octet-stream,1377793,BlockBlob
3,axavier/sampling/sample3.txt,"Fri, 13 Nov 2015 00:39:40 GMT",application/octet-stream,1377793,BlockBlob
4,axavier/sampling/sample4.txt,"Fri, 13 Nov 2015 00:41:49 GMT",application/octet-stream,1377793,BlockBlob


## Code python pour le reservoir sampling

In [6]:
import random
def reservoir_sampling(ensemble, k):
    N = len(ensemble)
    echantillon = []
    for i, e in enumerate(ensemble):
        if len(echantillon) < k:
            echantillon.append(k)
        else:
            j = random.randint(0, i)
            if j < k:
                echantillon[j] = e
    return echantillon

reservoir_sampling(ensemble, 10)

['6898i',
 '6868e',
 '7240m',
 '501h',
 '4124q',
 '9736m',
 '77z',
 '6668m',
 '9539x',
 '4440u']

## python à jython

## version distribuée

## version distribuée améliorée

## version java

On s'inspire de l'exemple suivant [Sampling](http://datafu.incubator.apache.org/docs/datafu/guide/sampling.html).
 On télécharge [datafu 1.2](http://datafu.incubator.apache.org/docs/datafu/) depuis [Maven](http://mvnrepository.com/artifact/com.linkedin.datafu/datafu/1.2.0). Ce n'est pas la dernière version mais suivre les instructions pour *builder* datafu (voir [documentation](http://datafu.incubator.apache.org/docs/datafu/1.2.0/)).

In [4]:
import pyensae
pyensae.download_data("datafu-1.2.0.jar", url="http://central.maven.org/maven2/com/linkedin/datafu/datafu/1.2.0/")

'datafu-1.2.0.jar'

In [47]:
%blob_up datafu-1.2.0.jar /$PSEUDO/sampling/datafu-1.2.0.jar

'$PSEUDO/sampling/datafu-1.2.0.jar'

In [153]:
%blob_ls /$PSEUDO/sampling

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/datafu-1.2.0.jar,"Fri, 13 Nov 2015 00:03:49 GMT",application/octet-stream,1600826,BlockBlob
1,axavier/sampling/sample.txt,"Fri, 13 Nov 2015 00:02:50 GMT",application/octet-stream,1377780,BlockBlob
2,axavier/sampling/sample2.txt,"Fri, 13 Nov 2015 00:35:55 GMT",application/octet-stream,1377793,BlockBlob
3,axavier/sampling/sample3.txt,"Fri, 13 Nov 2015 00:39:40 GMT",application/octet-stream,1377793,BlockBlob
4,axavier/sampling/sample4.txt,"Fri, 13 Nov 2015 00:41:49 GMT",application/octet-stream,1377793,BlockBlob


In [261]:
%%PIG_azure sample.pig

REGISTER '$CONTAINER/$PSEUDO/sampling/datafu-1.2.0.jar';

DEFINE RS datafu.pig.sampling.ReservoirSample('1000');

dset = LOAD '$CONTAINER/$PSEUDO/sampling/sample4.txt' 
        USING PigStorage('\t') AS (x:int, v:chararray) ;
sampled = FOREACH (GROUP dset ALL) GENERATE FLATTEN(RS(dset));
STORE sampled INTO '$CONTAINER/$PSEUDO/sampling/out_sampled_rs4_2015.txt' USING PigStorage() ;

In [262]:
jid = %hd_pig_submit sample.pig

In [265]:
st = %hd_job_status jid["id"]
st["id"],st["percentComplete"],st["completed"],st["status"]["jobComplete"],st["status"]["state"]

('job_1446540516812_0136', None, None, False, 'RUNNING')

In [266]:
%hd_tail_stderr jid["id"] -n 10

In [274]:
%blob_ls /$PSEUDO/sampling

,name,last_modified,content_type,content_length,blob_type
0,axavier/sampling/datafu-1.2.0.jar,"Fri, 13 Nov 2015 00:03:49 GMT",application/octet-stream,1600826,BlockBlob
1,axavier/sampling/out_sampled_rs4_2015.txt,"Fri, 13 Nov 2015 01:08:22 GMT",,0,BlockBlob
2,axavier/sampling/out_sampled_rs4_2015.txt/_SUC...,"Fri, 13 Nov 2015 01:08:22 GMT",application/octet-stream,0,BlockBlob
3,axavier/sampling/out_sampled_rs4_2015.txt/part...,"Fri, 13 Nov 2015 01:08:21 GMT",application/octet-stream,12785,BlockBlob
4,axavier/sampling/sample.txt,"Fri, 13 Nov 2015 00:02:50 GMT",application/octet-stream,1377780,BlockBlob
5,axavier/sampling/sample2.txt,"Fri, 13 Nov 2015 00:35:55 GMT",application/octet-stream,1377793,BlockBlob
6,axavier/sampling/sample3.txt,"Fri, 13 Nov 2015 00:39:40 GMT",application/octet-stream,1377793,BlockBlob
7,axavier/sampling/sample4.txt,"Fri, 13 Nov 2015 00:41:49 GMT",application/octet-stream,1377793,BlockBlob
8,axavier/sampling/sampled4_2015.txt,"Fri, 13 Nov 2015 00:50:20 GMT",,0,BlockBlob
9,axavier/sampling/sampled4_2015.txt/_SUCCESS,"Fri, 13 Nov 2015 00:50:20 GMT",application/octet-stream,0,BlockBlob


In [278]:
%blob_downmerge /$PSEUDO/sampling/out_sampled_rs4_2015.txt out_sampled_rs4_2015.txt -o

'out_sampled_rs4_2015.txt'

In [279]:
%head out_sampled_rs4_2015.txt

## fin

In [280]:
%blob_close

True